In [10]:
%load_ext autoreload
%load_ext watermark
%autoreload 2
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,tensorflow,keras

Mon Feb 11 2019 

CPython 3.6.8
IPython 7.2.0

numpy 1.15.4
scipy 1.2.0
sklearn not installed
pandas 0.23.4
tensorflow 1.12.0
keras 2.2.4

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-43-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


Using TensorFlow backend.


In [3]:
# TODO visualise and summarise the corpus data

In [7]:
from data.preprocess import PreProcessor

data = PreProcessor(config=PreProcessor.DEFAULT_CONFIG,
                    data_dir='data/raw/r252-corpus-features/org/elasticsearch/action/admin/cluster/allocation/')


In [11]:
len(data.metadata['token_vocab'])


401

In [21]:
from keras.models import Sequential
from keras import layers

embedding_dim = 128
vocabulary_size = len(data.metadata['token_vocab'])

model = Sequential()
model.add(layers.Embedding(vocabulary_size, embedding_dim, input_length=data.DEFAULT_CONFIG['max_chunk_length']))
model.add(layers.Conv1D(128, 1, activation='relu'))
model.add(layers.Conv1D(10, 1, activation='relu'))
model.add(layers.Conv1D(1, 1, activation='softmax'))
model.compile(optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 128)           51328     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 50, 128)           16512     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 50, 10)            1290      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 50, 1)             11        
Total params: 69,141
Trainable params: 69,141
Non-trainable params: 0
_________________________________________________________________


In [22]:
processed = data.get_tensorise_data()
code_snippet = processed['body_tokens']
label_name = processed['name_tokens']

In [23]:
# fit the model
model.fit(code_snippet, label_name, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(code_snippet, label_name, verbose=0)
print('Accuracy: %f' % (accuracy*100))

ValueError: Error when checking target: expected conv1d_5 to have 3 dimensions, but got array with shape (64, 50)

(64, 50)